In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from pathlib import Path

# -----------------------
# 1) Налаштування
# -----------------------
INPUT_CSV = "TrainingData.csv"
OUTPUT_CSV = "Output.csv"

# ВАШ список змінних-мітенів (як у R)
list_variables = [
    "THREAT_up",
    "THREAT_down",
    "citizen_impact",
    "PF_score",
    "PF_US",
    "PF_neg",
]

# Скільки перших рядків вважаємо train (як у R)
training_row = 300

# Кількість слів у словнику (як words.to.keep у R)
TOP_N = 200

# -----------------------
# 2) Завантаження та підготовка даних
# -----------------------
# Читаємо CSV
kdata = pd.read_csv(INPUT_CSV)

# Обираємо потрібні колонки (як у R)
base_cols = ["ID", "month", "year", "three_sentences"]
missing = [c for c in base_cols + list_variables if c not in kdata.columns]
if missing:
    raise ValueError(f"У файлі відсутні колонки: {missing}")

kdata = kdata[base_cols + list_variables].copy()

# Чистка тексту (нижній регістр; пунктуацію далі «з'їсть» токенізатор)
kdata["cleaned.body"] = (
    kdata["three_sentences"]
    .fillna("")
    .astype(str)
    .str.lower()
)

# Обчислюємо фактичний розмір train/test з урахуванням довжини датасету
n_docs = len(kdata)
if n_docs <= training_row:
    raise ValueError(
        f"У датасеті лише {n_docs} рядків — повинно бути > {training_row} для наявності тестової частини."
    )

train_idx = slice(0, training_row)
test_idx = slice(training_row, n_docs)

docs = kdata["cleaned.body"].tolist()

# -----------------------
# 3) Словник за документною частотою (аналог docfreq + top-200)
# -----------------------
# Спочатку лічильник присутності терміну в документі (binary=True)
count_vec = CountVectorizer(stop_words="english", binary=True)
X_bin = count_vec.fit_transform(docs)
vocab = np.array(count_vec.get_feature_names_out())

# Документна частота = сума по стовпцю (оскільки binary=True)
doc_freq = np.asarray(X_bin.sum(axis=0)).ravel()

# Топ-N слів за спаданням doc_freq
order = np.argsort(-doc_freq)
top_terms = vocab[order[: min(TOP_N, len(vocab))]]
top_terms_set = set(top_terms)

# -----------------------
# 4) TF-IDF за фіксованим словником top_terms (як dfm_tfidf у R на kdtm2)
# -----------------------
tfidf_vec = TfidfVectorizer(stop_words="english", vocabulary=sorted(top_terms_set))
X_tfidf = tfidf_vec.fit_transform(docs)  # fit на словнику, не на текстах (OK, словник фіксований)

# Матриці для train/test
X_train = X_tfidf[train_idx]
X_test = X_tfidf[test_idx]

# -----------------------
# 5) Навчання SVM і прогноз імовірностей для кожної цілі
# -----------------------
# Ініціалізуємо колонки для прогнозів як NaN
for target in list_variables:
    col_name = f"predicted.values_{target}"
    kdata[col_name] = np.nan

# Допоміжна функція: привести y до бінарних міток {0,1} як у R
def prepare_labels(y_raw):
    """
    - Порожні/NaN -> 0
    - Якщо значення виглядають як числа -> 0/1
    - Інакше робимо лейбл-енкодинг і попереджаємо, якщо більше 2 класів
    Повертає: y (np.array), name_of_positive_class (для вибору proba[:,1])
    """
    y = pd.Series(y_raw).copy()
    y = y.fillna(0)

    # якщо значення рядкові, спробуємо привести "0"/"1" до int
    def to_num(v):
        try:
            # прибираємо пробіли
            s = str(v).strip()
            # інколи можуть бути "0.0"/"1.0"
            return int(float(s))
        except:
            return v

    y = y.map(to_num)

    # Якщо вже {0,1}
    unique_vals = sorted(pd.Series(y).dropna().unique().tolist())
    if set(unique_vals).issubset({0,1}):
        return np.array(y, dtype=int), 1  # позитивний клас = 1

    # Інакше робимо лейбл-енкодинг
    le = LabelEncoder()
    y_enc = le.fit_transform(y.astype(str))
    classes = list(le.classes_)
    if len(classes) != 2:
        print(
            f"[УВАГА] Ціль має {len(classes)} клас(и): {classes}. "
            f"SVM з proba коректно працюватиме, але інтерпретація 'позитивного' класу умовна."
        )
    # позитивним вважаємо клас з індексом 1 (другий у лексикографічному порядку)
    positive_label_name = classes[1] if len(classes) >= 2 else classes[0]
    return y_enc, positive_label_name

# Тренуємо моделі
for target in list_variables:
    # y для train
    y_raw = kdata.loc[train_idx, target]
    y_train, positive_ref = prepare_labels(y_raw)

    # Модель SVM (аналог ksvm(..., kernel="rbfdot", C=50, prob.model=TRUE))
    clf = SVC(kernel="rbf", C=50, probability=True, random_state=42)

    # За аналогією з cross=10 у ksvm: обчислимо 10-fold CV-accuracy (опційно)
    try:
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        cv_scores = cross_val_score(clf, X_train, y_train, cv=skf, scoring="accuracy")
        print(f"[{target}] CV (10-fold) accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    except Exception as e:
        print(f"[{target}] Неможливо порахувати CV: {e}")

    # Навчання на всьому train і прогноз для test
    clf.fit(X_train, y_train)
    proba = clf.predict_proba(X_test)  # shape: (n_test, n_classes)

    # Вибираємо "другу" колонку як у R: predict(..., type="probabilities")[,2]
    # У sklearn другий стовпчик відповідає класу clf.classes_[1]
    # Якщо ми кодували у {0,1} — це буде proba класу 1 (тобто "позитивний")
    positive_class_index = 1 if proba.shape[1] > 1 else 0
    pos_proba = proba[:, positive_class_index]

    # Записуємо у колонку predicted.values_<target> тільки для тестової частини
    kdata.loc[test_idx, f"predicted.values_{target}"] = pos_proba

# -----------------------
# 6) Прибирання технічної колонки і збереження
# -----------------------
kdata = kdata.drop(columns=["cleaned.body"])
kdata.to_csv(OUTPUT_CSV, index=False)
print(f"Готово! Результат збережено у {Path(OUTPUT_CSV).resolve()}")


FileNotFoundError: [Errno 2] No such file or directory: 'TrainingData.csv'

In [2]:
!pip install scikit-learn


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 330.3 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 543.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
    --------------------------------------- 0.1/8.9 MB 4.3 MB/s eta 0:00:03
   - -------------------------------------- 0.3/8.9 MB 3.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/8.9 MB 5.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/8.9 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 1.9/8.9 MB 8.1 MB/s eta 0:00:01
   ----------- ---------------------------- 2.5/8.9 MB 8.8 MB/s eta 0:00:01
   ----------- ---------------------------- 2.5/8.9 MB 8.8 MB/s eta 0:00:01
   ------------ --------------------------- 2.8/8.9 MB 7.1 MB/s eta 0:00:01
   --------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\5103_6\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import os

# 1. Створюємо папку data
os.makedirs("data", exist_ok=True)

# 2. Вміст CSV
csv_content = """ID,month,year,three_sentences,THREAT_up,THREAT_down,citizen_impact,PF_score,PF_US,PF_neg
1,Jan,2025,"Threat is increasing in the north sector.",1,0,0,0,0,0
2,Jan,2025,"No threat detected in this area.",0,0,0,0,0,0
3,Jan,2025,"Citizens reported some impact on infrastructure.",0,0,1,0,0,0
4,Jan,2025,"Power failure reported in southern district.",0,0,0,0,0,0
5,Jan,2025,"No negative sentiment detected in reports.",0,0,0,0,0,1
6,Jan,2025,"US related issues are increasing according to data.",1,0,0,0,1,0
7,Jan,2025,"Threat level down in eastern front.",0,1,0,0,0,0
8,Jan,2025,"Citizen impact is minimal at this stage.",0,0,1,0,0,0
9,Jan,2025,"PF score shows moderate risk.",1,0,0,1,0,0
10,Jan,2025,"Everything seems calm, no events.",0,0,0,0,0,0
"""

# 3. Створюємо файл у папці data
with open("data/TrainingData.csv", "w", encoding="utf-8") as f:
    f.write(csv_content)

print("✅ Файл TrainingData.csv створено в папці data")


✅ Файл TrainingData.csv створено в папці data


In [6]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from pathlib import Path

# -----------------------
# 1) Налаштування
# -----------------------
INPUT_CSV = "data/TrainingData.csv"
OUTPUT_CSV = "Output.csv"

# Список цільових змінних
list_variables = ["THREAT_up", "THREAT_down", "citizen_impact", "PF_score", "PF_US", "PF_neg"]

# Скільки перших рядків беремо на тренування
training_row = 300
TOP_N = 200

# -----------------------
# 0) Якщо файлу немає – створюємо тестовий
# -----------------------
if not os.path.exists(INPUT_CSV):
    os.makedirs("data", exist_ok=True)
    csv_content = """ID,month,year,three_sentences,THREAT_up,THREAT_down,citizen_impact,PF_score,PF_US,PF_neg
1,Jan,2025,"Threat is increasing in the north sector.",1,0,0,0,0,0
2,Jan,2025,"No threat detected in this area.",0,0,0,0,0,0
3,Jan,2025,"Citizens reported some impact on infrastructure.",0,0,1,0,0,0
4,Jan,2025,"Power failure reported in southern district.",0,0,0,0,0,0
5,Jan,2025,"No negative sentiment detected in reports.",0,0,0,0,0,1
6,Jan,2025,"US related issues are increasing according to data.",1,0,0,0,1,0
7,Jan,2025,"Threat level down in eastern front.",0,1,0,0,0,0
8,Jan,2025,"Citizen impact is minimal at this stage.",0,0,1,0,0,0
9,Jan,2025,"PF score shows moderate risk.",1,0,0,1,0,0
10,Jan,2025,"Everything seems calm, no events.",0,0,0,0,0,0
"""
    with open(INPUT_CSV, "w", encoding="utf-8") as f:
        f.write(csv_content)
    print(f"⚠️  Файл {INPUT_CSV} не знайдено — створено тестовий приклад із 10 рядками.")

# -----------------------
# 2) Завантаження даних
# -----------------------
kdata = pd.read_csv(INPUT_CSV)

# Автоматично підлаштовуємо training_row якщо рядків менше 300
if len(kdata) < training_row:
    training_row = len(kdata) // 2
    print(f"⚠️  training_row зменшено до {training_row}, бо рядків у файлі лише {len(kdata)}.")

base_cols = ["ID", "month", "year", "three_sentences"]
kdata = kdata[base_cols + list_variables].copy()

# Чистимо текст
kdata["cleaned.body"] = kdata["three_sentences"].fillna("").astype(str).str.lower()

docs = kdata["cleaned.body"].tolist()
n_docs = len(docs)
train_idx = slice(0, training_row)
test_idx = slice(training_row, n_docs)

# -----------------------
# 3) Топ-200 термінів за document frequency
# -----------------------
count_vec = CountVectorizer(stop_words="english", binary=True)
X_bin = count_vec.fit_transform(docs)
vocab = np.array(count_vec.get_feature_names_out())
doc_freq = np.asarray(X_bin.sum(axis=0)).ravel()

order = np.argsort(-doc_freq)
top_terms = vocab[order[:min(TOP_N, len(vocab))]]
tfidf_vec = TfidfVectorizer(stop_words="english", vocabulary=sorted(set(top_terms)))
X_tfidf = tfidf_vec.fit_transform(docs)

X_train = X_tfidf[train_idx]
X_test = X_tfidf[test_idx]

# -----------------------
# 4) Навчання моделей
# -----------------------
for target in list_variables:
    y = kdata.loc[train_idx, target].fillna(0).astype(int)

    clf = SVC(kernel="rbf", C=50, probability=True, random_state=42)
    if len(set(y)) > 1:
        skf = StratifiedKFold(n_splits=min(5, len(set(y))*2), shuffle=True, random_state=42)
        scores = cross_val_score(clf, X_train, y, cv=skf, scoring="accuracy")
        print(f"[{target}] CV accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

    clf.fit(X_train, y)
    if X_test.shape[0] > 0:
        probs = clf.predict_proba(X_test)[:, 1]  # ймовірність класу 1
        kdata.loc[test_idx, f"predicted.values_{target}"] = probs
    else:
        kdata[f"predicted.values_{target}"] = np.nan

kdata = kdata.drop(columns=["cleaned.body"])
kdata.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Готово! Результат збережено у {Path(OUTPUT_CSV).resolve()}")


⚠️  training_row зменшено до 5, бо рядків у файлі лише 10.


ValueError: Found input variables with inconsistent numbers of samples: [5, 6]

In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC

# -----------------------
# 1) Налаштування
# -----------------------
INPUT_CSV = "data/TrainingData.csv"
OUTPUT_CSV = "Output.csv"
LIST_VARIABLES = ["THREAT_up", "THREAT_down", "citizen_impact", "PF_score", "PF_US", "PF_neg"]
TOP_N = 200          # скільки термінів лишити за docfreq
DEFAULT_TRAIN = 300  # як у R-скрипті

# -----------------------
# 0) Автогенерація тестових даних, якщо файлу немає
# -----------------------
def ensure_sample_csv(path: str):
    if os.path.exists(path):
        return
    os.makedirs(os.path.dirname(path), exist_ok=True)
    csv_content = """ID,month,year,three_sentences,THREAT_up,THREAT_down,citizen_impact,PF_score,PF_US,PF_neg
1,Jan,2025,"Threat is increasing in the north sector.",1,0,0,0,0,0
2,Jan,2025,"No threat detected in this area.",0,0,0,0,0,0
3,Jan,2025,"Citizens reported some impact on infrastructure.",0,0,1,0,0,0
4,Jan,2025,"Power failure reported in southern district.",0,0,0,0,0,0
5,Jan,2025,"No negative sentiment detected in reports.",0,0,0,0,0,1
6,Jan,2025,"US related issues are increasing according to data.",1,0,0,0,1,0
7,Jan,2025,"Threat level down in eastern front.",0,1,0,0,0,0
8,Jan,2025,"Citizen impact is minimal at this stage.",0,0,1,0,0,0
9,Jan,2025,"PF score shows moderate risk.",1,0,0,1,0,0
10,Jan,2025,"Everything seems calm, no events.",0,0,0,0,0,0
"""
    with open(path, "w", encoding="utf-8") as f:
        f.write(csv_content)
    print(f"⚠️  {path} не знайдено — створено тестовий файл із 10 рядками.")

ensure_sample_csv(INPUT_CSV)

# -----------------------
# 2) Завантаження та базова підготовка
# -----------------------
df = pd.read_csv(INPUT_CSV)
base_cols = ["ID", "month", "year", "three_sentences"]

missing_cols = [c for c in base_cols + LIST_VARIABLES if c not in df.columns]
if missing_cols:
    raise ValueError(f"У файлі відсутні необхідні колонки: {missing_cols}")

df = df[base_cols + LIST_VARIABLES].copy()
df["cleaned.body"] = df["three_sentences"].fillna("").astype(str).str.lower()

n_docs = len(df)
if n_docs < 2:
    raise ValueError("Потрібно щонайменше 2 рядки (один для train, один для test).")

# training_row: якщо мало рядків — беремо половину (мінімум 1)
training_row = min(DEFAULT_TRAIN, max(1, n_docs // 2))
train_idx = slice(0, training_row)
test_idx = slice(training_row, n_docs)

docs = df["cleaned.body"].tolist()

# -----------------------
# 3) Словник за document frequency → TF-IDF
# -----------------------
count_vec = CountVectorizer(stop_words="english", binary=True)
X_bin = count_vec.fit_transform(docs)
vocab = np.array(count_vec.get_feature_names_out())

if vocab.size == 0:
    raise ValueError("Після токенізації словник порожній. Перевірте тексти або вимкніть стоп-слова.")

doc_freq = np.asarray(X_bin.sum(axis=0)).ravel()
order = np.argsort(-doc_freq)
top_terms = vocab[order[: min(TOP_N, vocab.size)]]
vocab_fixed = sorted(set(top_terms))

tfidf_vec = TfidfVectorizer(stop_words="english", vocabulary=vocab_fixed)
X_tfidf = tfidf_vec.fit_transform(docs)

X_train = X_tfidf[train_idx]
X_test = X_tfidf[test_idx]

print(f"📊 Docs: {n_docs} | Train: {X_train.shape[0]} | Test: {X_test.shape[0]} | Vocab: {len(vocab_fixed)}")

# -----------------------
# 4) Допоміжні функції
# -----------------------
def to_binary_labels(series: pd.Series) -> np.ndarray:
    """
    Перетворює ціль у бінарні мітки {0,1}.
    НЧ/порожні → 0. Будь-що не 0 → 1.
    """
    y = pd.to_numeric(series, errors="coerce").fillna(0)
    y = (y != 0).astype(int)
    return y.to_numpy()

def safe_cv(clf, X, y) -> None:
    """
    Обчислює k-fold CV тільки якщо вистачає даних у найменшому класі.
    """
    unique, counts = np.unique(y, return_counts=True)
    class_counts = dict(zip(unique.tolist(), counts.tolist()))
    if len(unique) < 2:
        print(f"• CV пропущено: лише один клас у train. Класи: {class_counts}")
        return
    min_class = counts.min()
    n_train = X.shape[0]
    # Маємо мати щонайменше 2 зразки в мінімальному класі і ≥4 загалом
    if min_class < 2 or n_train < 4:
        print(f"• CV пропущено: замало даних (min_class={min_class}, n_train={n_train}). Класи: {class_counts}")
        return
    n_splits = min(5, int(min_class))
    if n_splits < 2:
        print(f"• CV пропущено: n_splits={n_splits} < 2.")
        return
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(clf, X, y, cv=skf, scoring="accuracy")
    print(f"• CV ({n_splits}-fold) accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

# -----------------------
# 5) Навчання моделей і прогнози
# -----------------------
for target in LIST_VARIABLES:
    print(f"\n🧠 Модель для цілі: {target}")
    y_train = to_binary_labels(df.loc[train_idx, target])

    # Перевірка узгодженості
    if X_train.shape[0] != len(y_train):
        raise ValueError(
            f"Несумісні розміри: X_train={X_train.shape[0]} vs y_train={len(y_train)}. "
            f"Перевірте формування train_idx/test_idx."
        )

    clf = SVC(kernel="rbf", C=50, probability=True, random_state=42)

    # Безпечний CV (за наявності достатніх даних)
    safe_cv(clf, X_train, y_train)

    # Навчання на всьому train
    clf.fit(X_train, y_train)

    # Прогнози для тесту (ймовірність класу 1)
    col_name = f"predicted.values_{target}"
    df[col_name] = np.nan
    if X_test.shape[0] > 0:
        proba = clf.predict_proba(X_test)
        # індекс класу "1" (якщо чомусь його немає — беремо останню колонку)
        classes = clf.classes_
        pos_index = int(np.where(classes == 1)[0][0]) if 1 in classes else (proba.shape[1] - 1)
        df.loc[test_idx, col_name] = proba[:, pos_index]
    else:
        print("• Тестових прикладів немає — прогноз пропущено.")

# -----------------------
# 6) Збереження результату
# -----------------------
out_df = df.drop(columns=["cleaned.body"])
out_path = Path(OUTPUT_CSV).resolve()
out_df.to_csv(out_path, index=False, encoding="utf-8")
print(f"\n✅ Готово! Результат збережено у {out_path}")


📊 Docs: 10 | Train: 5 | Test: 5 | Vocab: 33

🧠 Модель для цілі: THREAT_up


ValueError: Несумісні розміри: X_train=5 vs y_train=6. Перевірте формування train_idx/test_idx.